In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

idx = pd.IndexSlice

# CRISP-DM Flow

## Business Understanding

## Data Understanding

## Data Preparation

## Data Modeling

## Result Evaluation

## Deployment